In [1]:
#step 1 https://pycistopic.readthedocs.io/en/latest/notebooks/human_cerebellum.html#
#maybe its the multiplexed data - why not try one multiome sample from NPOD1
#/nfs/lab/projects/nPOD/cellranger_output/Multiome/MM_660/outs
import os
os.chdir("/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/T1D_early/")

In [2]:
import pycisTopic
pycisTopic.__version__

'2.0a0'

In [3]:
import os
out_dir = "outs"
os.makedirs(out_dir, exist_ok = True)

In [4]:
fragments_dict = {
    "MM_661": "/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_661/outs/atac_fragments.tsv.gz",
    "MM_665": "/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_665/outs/atac_fragments.tsv.gz"
}

In [5]:
#this was created from ripping the metadata from the RDS object using the script /tscc/projects/ps-gaultonlab/abhowell/scenic_data/create_npod2_meta_forscenicplus.R
#metadata comes from original RDS object
#/tscc/projects/ps-gaultonlab/abhowell/scenic_data/031424_PancGRS_8lanes_merged_filt5perMito_100katacRNAcounts_peakCallLieden_celltypesLabelled_wlymph_rmLowQualAcinar_peakCallCelltype_acinarSubtypes_usingNPOD1refmap_newWindowsAssayNoCHR_finalPeakCalls_wGeneAct.rds
import pandas as pd
cell_data = pd.read_table("npod1_T1Dearly_metadata.tsv", index_col = 0)
cell_data.head()

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,RNA_snn_res.1,seurat_clusters,samples,sex,condition,RNA_snn_res.0.5,...,SeuratClusters,celltype_assignment2,barcode,nPOD_barcode,CurrentObjLabels,nPOD_ID,technology,nCount_RAW_mtx,nFeature_RAW_mtx,barcode2
AAACCAACAAACCTTG-1-MM_665,SeuratProject,3105,1640,0.000000,2,4,MM_665,M,T1D,4,...,4_2,Ductal,AAACCAACAAACCTTG-1,MM_665_AAACCAACAAACCTTG,Ductal,multi_6228,multiome,3251,1728,AAACCAACAAACCTTG-1-MM_665
AAACCGGCACCATATG-1-MM_665,SeuratProject,6813,2641,0.000000,3,3,MM_665,M,T1D,3,...,3,Acinar,AAACCGGCACCATATG-1,MM_665_AAACCGGCACCATATG,Acinar_3,multi_6228,multiome,7576,3021,AAACCGGCACCATATG-1-MM_665
AAACGGATCAGCTAAC-1-MM_665,SeuratProject,4452,2028,0.202156,2,4,MM_665,M,T1D,4,...,4_7,Ductal,AAACGGATCAGCTAAC-1,MM_665_AAACGGATCAGCTAAC,Ductal,multi_6228,multiome,4723,2186,AAACGGATCAGCTAAC-1-MM_665
AAACGTACACCTATAG-1-MM_665,SeuratProject,4373,1971,0.000000,3,3,MM_665,M,T1D,3,...,3,Acinar,AAACGTACACCTATAG-1,MM_665_AAACGTACACCTATAG,Acinar_3,multi_6228,multiome,4694,2142,AAACGTACACCTATAG-1-MM_665
AAAGCCGCAACCTGGT-1-MM_665,SeuratProject,2748,1517,0.000000,3,3,MM_665,M,T1D,3,...,3,Acinar,AAAGCCGCAACCTGGT-1,MM_665_AAAGCCGCAACCTGGT,Acinar_3,multi_6228,multiome,2892,1608,AAAGCCGCAACCTGGT-1-MM_665


In [6]:
chromsizes = pd.read_table(
    "http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes",
    header = None,
    names = ["Chromosome", "End"]
)
chromsizes.insert(1, "Start", 0)
chromsizes.head()

,Chromosome,Start,End
0,chr1,0,248956422
1,chr2,0,242193529
2,chr3,0,198295559
3,chr4,0,190214555
4,chr5,0,181538259


In [7]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
os.makedirs(os.path.join(out_dir, "consensus_peak_calling"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"), exist_ok = True)

/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 19:51:13,645	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:
bw_paths, bed_paths = export_pseudobulk(
    input_data = cell_data,
    variable = "celltype_assignment2",
    sample_id_col = "samples",
    chromsizes = chromsizes,
    bed_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bed_files"),
    bigwig_path = os.path.join(out_dir, "consensus_peak_calling/pseudobulk_bw_files"),
    path_to_fragments = fragments_dict,
    n_cpu = 20,
    normalize_bigwig = True,
    temp_dir = "/tmp",
    split_pattern = "-")

2025-03-06 19:51:29,680 cisTopic     INFO     Splitting fragments by cell type.
2025-03-06 19:53:02,583 cisTopic     INFO     generating bigwig files


In [9]:
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv"), "wt") as f:
    for v in bw_paths:
        _ = f.write(f"{v}\t{bw_paths[v]}\n")

In [10]:
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv"), "wt") as f:
    for v in bed_paths:
        _ = f.write(f"{v}\t{bed_paths[v]}\n")

In [11]:
bw_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bw_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bw_paths.update({v: p})

In [12]:
bed_paths = {}
with open(os.path.join(out_dir, "consensus_peak_calling/bed_paths.tsv")) as f:
    for line in f:
        v, p = line.strip().split("\t")
        bed_paths.update({v: p})

In [13]:
from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path = "macs2"

os.makedirs(os.path.join(out_dir, "consensus_peak_calling/MACS"), exist_ok = True)

narrow_peak_dict = peak_calling(
    macs_path = macs_path,
    bed_paths = bed_paths,
    outdir = os.path.join(os.path.join(out_dir, "consensus_peak_calling/MACS")),
    genome_size = 'hs',
    n_cpu = 10,
    input_format = 'BEDPE',
    shift = 73,
    ext_size = 146,
    keep_dup = 'all',
    q_value = 0.05,
    _temp_dir = '/tscc/lustre/ddn/scratch/abhowell'
)

2025-03-06 19:54:50,944	INFO worker.py:1724 -- Started a local Ray instance.


(macs_call_peak_ray pid=1642777) 2025-03-06 19:54:53,756 cisTopic     INFO     Calling peaks for Macrophage with macs2 callpeak --treatment outs/consensus_peak_calling/pseudobulk_bed_files/Macrophage.fragments.tsv.gz --name Macrophage  --outdir outs/consensus_peak_calling/MACS --format BEDPE --gsize hs --qvalue 0.05 --nomodel --shift 73 --extsize 146 --keep-dup all --call-summits --nolambda
(macs_call_peak_ray pid=1642780) 2025-03-06 19:54:53,756 cisTopic     INFO     Calling peaks for Endothelial with macs2 callpeak --treatment outs/consensus_peak_calling/pseudobulk_bed_files/Endothelial.fragments.tsv.gz --name Endothelial  --outdir outs/consensus_peak_calling/MACS --format BEDPE --gsize hs --qvalue 0.05 --nomodel --shift 73 --extsize 146 --keep-dup all --call-summits --nolambda
(macs_call_peak_ray pid=1642775) 2025-03-06 19:54:53,756 cisTopic     INFO     Calling peaks for Activated_stellate with macs2 callpeak --treatment outs/consensus_peak_calling/pseudobulk_bed_files/Activated_st

In [14]:
#had to clone pycistopic github seperately - not in scenic+ git clone
from pycisTopic.iterative_peak_calling import get_consensus_peaks
# Other param
peak_half_width=250
path_to_blacklist="/tscc/nfs/home/abhowell/pycisTopic/blacklist/hg38-blacklist.v2.bed"
# Get consensus peaks
consensus_peaks = get_consensus_peaks(
    narrow_peaks_dict = narrow_peak_dict,
    peak_half_width = peak_half_width,
    chromsizes = chromsizes,
    path_to_blacklist = path_to_blacklist)

2025-03-06 19:57:38,507 cisTopic     INFO     Extending and merging peaks per class
2025-03-06 19:58:22,066 cisTopic     INFO     Normalizing peak scores
2025-03-06 19:58:22,368 cisTopic     INFO     Merging peaks
Warning! Start and End columns now have different dtypes: int64 and int32
2025-03-06 19:59:00,533 cisTopic     INFO     Done!


In [15]:
#this gives a warning but it does make the file consensus_regions.bed so its fine
consensus_peaks.to_bed(
    path = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed"),
    keep =True,
    compression = 'infer',
    chain = False)

/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  return pd.concat([outdf, df.get(noncanonical)], axis=1)
/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pyranges/out.py:37: FutureWarning: Passing a set as an indexer is deprecated an

In [16]:
!mkdir -p outs/qc/

In [17]:
regions_bed_filename = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
tss_bed_filename = os.path.join(out_dir, "qc", "tss.bed")

pycistopic_qc_commands_filename = "pycistopic_qc_commands.txt"

# Create text file with all pycistopic qc command lines.
with open(pycistopic_qc_commands_filename, "w") as fh:
    for sample, fragment_filename in fragments_dict.items():
        print(
            "pycistopic qc run",
            f"--fragments {fragment_filename}",
            f"--regions {regions_bed_filename}",
            f"--tss /tscc/projects/ps-gaultonlab/abhowell/scenic_data_npod1/qc/tss.bed",
            f"--output /tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/T1D_early/outs/qc/{sample}",
            sep=" ",
            file=fh
        )

In [18]:
#add to this file
##!/bin/bash
#SBATCH --partition=condo
#SBATCH --qos=condo
#SBACTH --nodes=4
#SBATCH --ntasks-per-node=1
#SBATCH --time=3-00:00:00
#SBATCH --account=csd854
#SBATCH --mem=300GB

#conda activate pycistopic_polars

In [8]:
import os
out_dir = "outs"
os.makedirs(out_dir, exist_ok = True)

from pycisTopic.qc import get_barcodes_passing_qc_for_sample
#sample_id_to_barcodes_passing_filters = {}
#sample_id_to_thresholds = {}

#Nuclei with less than
##849 1000 unique ATAC-seq fragments were removed. Initial quality control was performed to retain
#850 cells in each sample using the following metrics unique usable reads > 5000, fraction promoters
#851 used > 0.01, TSS enrichment (TSSe) > 0.3 using scanPy v1.8.0. Doublets were removed using
#Amulet v1.0 per sample70


#for sample_id in fragments_dict:
#    (
#        sample_id_to_barcodes_passing_filters[sample_id],
#        sample_id_to_thresholds[sample_id]
#    ) = get_barcodes_passing_qc_for_sample(
#            sample_id = sample_id,
#            pycistopic_qc_output_dir = "outs/qc",
            #unique_fragments_threshold = 5000, # use automatic thresholding
            #tss_enrichment_threshold = 0.3, # use automatic thresholding
            #frip_threshold = 0.01,
#            use_automatic_thresholds = False,
#    )

In [ ]:
#actual creating of the cistopic object
path_to_regions = os.path.join(out_dir, "consensus_peak_calling/consensus_regions.bed")
path_to_blacklist="/tscc/nfs/home/abhowell/pycisTopic/blacklist/hg38-blacklist.v2.bed"
pycistopic_qc_output_dir = "outs/qc"

from pycisTopic.cistopic_class import create_cistopic_object_from_fragments
import polars as pl

#cistopic_obj = create_cistopic_object(fragment_matrix=count_matrix, path_to_blacklist=path_to_blacklist)


cistopic_obj_list = []
for sample_id in fragments_dict:
   # sample_metrics = pl.read_parquet(
   #     os.path.join(pycistopic_qc_output_dir, f'{sample_id}.fragments_stats_per_cb.parquet')
   # ).to_pandas().set_index("CB").loc[ sample_id_to_barcodes_passing_filters[sample_id] ]
    cistopic_obj = create_cistopic_object_from_fragments(
        path_to_fragments = fragments_dict[sample_id],
        path_to_regions = path_to_regions,
        path_to_blacklist = path_to_blacklist,
    #    metrics = sample_metrics,
        valid_bc = sample_id_to_barcodes_passing_filters[sample_id],
        n_cpu = 40,
        project = sample_id,
        split_pattern = '-'
    )
    cistopic_obj_list.append(cistopic_obj)

2025-03-06 21:12:39,843 cisTopic     INFO     Reading data for MM_661
2025-03-06 21:13:42,386 cisTopic     INFO     Counting number of unique fragments (Unique_nr_frag)
2025-03-06 21:14:10,667 cisTopic     INFO     Counting fragments in regions


/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 21:14:10,994	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-06 21:14:13,203	INFO worker.py:1724 -- Started a local Ray instance.


2025-03-06 21:14:36,622 cisTopic     INFO     Creating fragment matrix


/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pycisTopic/cistopic_class.py:886: PerformanceWarning: The following operation may generate 149808982500 cells in the resulting pandas object.
  .unstack(level="Name", fill_value=0)


2025-03-06 21:16:37,836 cisTopic     INFO     Data is too big, making partitions. This is a reported error in Pandas versions > 0.21 (https://github.com/pandas-dev/pandas/issues/26314)


...

/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pycisTopic/cistopic_class.py:954: PerformanceWarning: The following operation may generate 29865516758 cells in the resulting pandas object.
  .unstack(level="Name", fill_value=0)


2025-03-06 21:38:22,933 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-03-06 21:43:07,170 cisTopic     INFO     Removing blacklisted regions
2025-03-06 21:43:07,922 cisTopic     INFO     Creating CistopicObject
2025-03-06 21:43:09,810 cisTopic     INFO     Done!


/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pycisTopic/cistopic_class.py:954: PerformanceWarning: The following operation may generate 29888415340 cells in the resulting pandas object.
  .unstack(level="Name", fill_value=0)


2025-03-06 21:47:23,836 cisTopic     INFO     Converting fragment matrix to sparse matrix
2025-03-06 21:51:30,791 cisTopic     INFO     Removing blacklisted regions
2025-03-06 21:51:31,685 cisTopic     INFO     Creating CistopicObject
2025-03-06 21:51:33,450 cisTopic     INFO     Done!


/tscc/nfs/home/abhowell/miniconda3/envs/pycistopic_polars/lib/python3.11/site-packages/pycisTopic/cistopic_class.py:954: PerformanceWarning: The following operation may generate 29856044323 cells in the resulting pandas object.
  .unstack(level="Name", fill_value=0)


In [ ]:
import pandas as pd
cell_data = pd.read_table("npod1_T1Dearly_metadata.tsv", index_col = 0)
cell_data.head()
cistopic_obj.add_cell_data(cell_data)

In [ ]:
cistopic_obj.cell_data
counts = cistopic_obj.cell_data.celltype_assignment2.value_counts()
print(counts)

In [ ]:
import pickle
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

In [ ]:
#import scrublet as scr
#scrub = scr.Scrublet(cistopic_obj.fragment_matrix.T, expected_doublet_rate=0.1)
#doublet_scores, predicted_doublets = scrub.scrub_doublets()
#scrub.plot_histogram();
#scrub.call_doublets(threshold=0.22)
#scrub.plot_histogram();
#scrublet = pd.DataFrame([scrub.doublet_scores_obs_, scrub.predicted_doublets_], columns=cistopic_obj.cell_names, index=['Doublet_scores_fragments', 'Predicted_doublets_fragments']).T

In [ ]:
#cistopic_obj.add_cell_data(scrublet, split_pattern = '-')
#sum(cistopic_obj.cell_data.Predicted_doublets_fragments == True)

In [ ]:
#pickle.dump(
#    cistopic_obj,
#    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
#)

In [ ]:
# Remove doublets
#singlets = cistopic_obj.cell_data[cistopic_obj.cell_data.Predicted_doublets_fragments == False].index.tolist()
# Subset cisTopic object
#cistopic_obj_noDBL = cistopic_obj.subset(singlets, copy=True, split_pattern='-')
#print(cistopic_obj_noDBL)

In [ ]:
#pickle.dump(
#    cistopic_obj,
#    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
#)

In [ ]:
#!wget https://github.com/mimno/Mallet/releases/download/v202108/Mallet-202108-bin.tar.gz
#!tar -xf Mallet-202108-bin.tar.gz

In [ ]:
!mkdir -p mallet/

In [ ]:
os.environ['MALLET_MEMORY'] = '400G'
from pycisTopic.lda_models import run_cgs_models_mallet
# Configure path Mallet
mallet_path="/tscc/projects/ps-gaultonlab/abhowell/scenic_data_npod1/Mallet-202108/bin/mallet"
# Run models
models=run_cgs_models_mallet(
    cistopic_obj,
    n_topics=[2, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    n_cpu=12,
    n_iter=500,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    tmp_path="/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/T1D_early/mallet",
    save_path="/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/T1D_early/mallet",
    mallet_path=mallet_path,
)

In [ ]:
pickle.dump(
    models,
    open(os.path.join(out_dir, "models.pkl"), "wb")
)

In [ ]:
from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 40,
    return_model = True
)

In [ ]:
cistopic_obj.add_LDA_model(model)

In [ ]:
pickle.dump(
    cistopic_obj,
    open(os.path.join(out_dir, "cistopic_obj.pkl"), "wb")
)

In [ ]:
from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

In [ ]:
find_clusters(
    cistopic_obj,
    target  = 'cell',
    k = 10,
    res = [0.6, 1.2, 3],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = '-'
)

In [ ]:
plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['celltype_assignment2', 'pycisTopic_leiden_10_0.6', 'pycisTopic_leiden_10_1.2', 'pycisTopic_leiden_10_3'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
annot_dict = {}
for resolution in [0.6, 1.2, 3]:
    annot_dict[f"pycisTopic_leiden_10_{resolution}"] = {}
    for cluster in set(cistopic_obj.cell_data[f"pycisTopic_leiden_10_{resolution}"]):
        counts = cistopic_obj.cell_data.loc[
            cistopic_obj.cell_data.loc[cistopic_obj.cell_data[f"pycisTopic_leiden_10_{resolution}"] == cluster].index,
            "celltype_assignment2"].value_counts()
        annot_dict[f"pycisTopic_leiden_10_{resolution}"][cluster] = f"{counts.index[counts.argmax()]}({cluster})"

In [ ]:
for resolution in [0.6, 1.2, 3]:
    cistopic_obj.cell_data[f'pycisTopic_leiden_10_{resolution}'] = [
        annot_dict[f'pycisTopic_leiden_10_{resolution}'][x] for x in cistopic_obj.cell_data[f'pycisTopic_leiden_10_{resolution}'].tolist()
    ]

In [ ]:
plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['celltype_assignment2', 'pycisTopic_leiden_10_0.6', 'pycisTopic_leiden_10_1.2', 'pycisTopic_leiden_10_3'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

In [ ]:
plot_topic(
    cistopic_obj,
    reduction_name = 'UMAP',
    target = 'cell',
    num_columns=5
)

In [ ]:
from pycisTopic.topic_binarization import binarize_topics

In [ ]:
region_bin_topics_top_3k = binarize_topics(
    cistopic_obj, method='ntop', ntop = 3_000,
    plot=True, num_columns=5
)

In [ ]:
region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=True, num_columns=5
)

In [ ]:
binarized_cell_topic = binarize_topics(
    cistopic_obj,
    target='cell',
    method='li',
    plot=True,
    num_columns=5, nbins=100)

In [ ]:
from pycisTopic.topic_qc import compute_topic_metrics, plot_topic_qc, topic_annotation
import matplotlib.pyplot as plt
from pycisTopic.utils import fig2img

In [ ]:
topic_qc_metrics = compute_topic_metrics(cistopic_obj)

In [ ]:
fig_dict={}
fig_dict['CoherenceVSAssignments']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Log10_Assignments', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['AssignmentsVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Log10_Assignments', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSCells_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Cells_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSRegions_in_bin']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Regions_in_binarized_topic', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSMarginal_dist']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Marginal_topic_dist', var_color='Gini_index', plot=False, return_fig=True)
fig_dict['CoherenceVSGini_index']=plot_topic_qc(topic_qc_metrics, var_x='Coherence', var_y='Gini_index', var_color='Gini_index', plot=False, return_fig=True)

In [ ]:
# Plot topic stats in one figure
fig=plt.figure(figsize=(40, 43))
i = 1
for fig_ in fig_dict.keys():
    plt.subplot(2, 3, i)
    img = fig2img(fig_dict[fig_]) #To convert figures to png to plot together, see .utils.py. This converts the figure to png.
    plt.imshow(img)
    plt.axis('off')
    i += 1
plt.subplots_adjust(wspace=0, hspace=-0.70)
plt.show()

In [ ]:
topic_annot = topic_annotation(
    cistopic_obj,
    annot_var='celltype_assignment2',
    binarized_cell_topic=binarized_cell_topic,
    general_topic_thr = 0.2
)

In [ ]:
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
import numpy as np

In [ ]:
imputed_acc_obj = impute_accessibility(
    cistopic_obj,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

In [ ]:
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)

In [ ]:
variable_regions = find_highly_variable_features(
    normalized_imputed_acc_obj,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True
)

In [ ]:
markers_dict= find_diff_features(
    cistopic_obj,
    imputed_acc_obj,
    variable='celltype_assignment2',
    var_features=variable_regions,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=5,
    _temp_dir='/tmp',
    split_pattern = '-'
)

In [ ]:
from pycisTopic.clust_vis import plot_imputed_features

In [ ]:
#threw an error when I tried to include Alpha because if you run markers_dict["Alpha"] there are no results
#you can fix this by changing features list
#features=[markers_dict[x].index.tolist()[0] for x in ['Beta', 'T_cells']],

#markers_dict["Beta"]

In [ ]:
#plot_imputed_features(
#    cistopic_obj,
#    reduction_name='UMAP',
#    imputed_data=imputed_acc_obj,
#    features=[markers_dict[x].index.tolist()[0] for x in ['Alpha', 'T_cells']],
#    scale=False,
#    num_columns=1
#)

In [ ]:
print("Number of DARs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(markers_dict[x])}")

In [ ]:
os.makedirs(os.path.join(out_dir, "region_sets"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_otsu"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "Topics_top_3k"), exist_ok = True)
os.makedirs(os.path.join(out_dir, "region_sets", "DARs_cell_type"), exist_ok = True)

In [ ]:
from pycisTopic.utils import region_names_to_coordinates

In [ ]:
for topic in region_bin_topics_otsu:
    region_names_to_coordinates(
        region_bin_topics_otsu[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_otsu", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
for cell_type in markers_dict:
    region_names_to_coordinates(
        markers_dict[cell_type].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "DARs_cell_type", f"{cell_type}.bed"),
        sep = "\t",
        header = False, index = False
    )

In [ ]:
import pyranges as pr
from pycisTopic.gene_activity import get_gene_activity

In [ ]:
chromsizes = pd.read_table("/tscc/projects/ps-gaultonlab/abhowell/scenic_data_npod1/qc/hg38.chrom_sizes_and_alias.tsv")
chromsizes

In [ ]:
chromsizes.rename({"# ucsc": "Chromosome", "length": "End"}, axis = 1, inplace = True)
chromsizes["Start"] = 0
chromsizes = pr.PyRanges(chromsizes[["Chromosome", "Start", "End"]])

In [ ]:
pr_annotation = pd.read_table(
        os.path.join("/tscc/projects/ps-gaultonlab/abhowell/scenic_data_npod1/qc/tss.bed")
    ).rename(
        {"Name": "Gene", "# Chromosome": "Chromosome"}, axis = 1)
pr_annotation["Transcription_Start_Site"] = pr_annotation["Start"]
pr_annotation = pr.PyRanges(pr_annotation)
pr_annotation

In [ ]:
gene_act, weigths = get_gene_activity(
    imputed_acc_obj,
    pr_annotation,
    chromsizes,
    use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
    upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it
                             # these bp will be taken (1kbp here)
    downstream=[1000,100000], # Search space downstream
    distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
    decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
    extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for
                          #this weight)
    extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
    gene_size_weight=False, # Whether to add a weights based on the length of the gene
    gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                          #in the genome
    remove_promoters=False, # Whether to remove promoters when computing gene activity scores
    average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene
                          #activity score
    scale_factor=1, # Value to multiply for the final gene activity matrix
    extend_tss=[10,10], # Space to consider a promoter
    gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
    return_weights= True, # Whether to return the final weights
    project='Gene_activity') # Project name for the gene activity object

In [ ]:
DAG_markers_dict= find_diff_features(
    cistopic_obj,
    gene_act,
    variable='celltype_assignment2',
    var_features=None,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.5),
    n_cpu=5,
    _temp_dir='/tmp',
    split_pattern = '-')

In [ ]:
print("Number of DAGs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(DAG_markers_dict[x])}")

In [ ]:
#leaving off just before "Label Transfer" step

In [ ]:
import scanpy as sc

In [ ]:
#adata = sc.read_10x_mtx(
#    "/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_661/outs/filtered_feature_bc_matrix/",
##    var_names = "gene_symbols"
#)


samples = {
    "MM_665": "/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_665/outs/filtered_feature_bc_matrix/",
    "MM_661": "/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_661/outs/filtered_feature_bc_matrix/",
}
adatas = {}

for sample_id, filename in samples.items():
    path = filename
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()
print(adata.obs["sample"].value_counts())
adata

In [ ]:
adata.var_names_make_unique()

In [ ]:
import pandas as pd
cell_data = pd.read_table("/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_661/npod1_MM_661_metadata.tsv", index_col = 0)
cell_data.head()

In [ ]:
cell_data.index

In [ ]:
adata.obs_names

In [ ]:
adata = adata[list(set(adata.obs_names) & set(cell_data.index))].copy()

In [ ]:
adata.obs = cell_data.loc[adata.obs_names]


In [ ]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)

In [ ]:
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata)
sc.pl.pca(adata, color = "celltype_assignment2")

In [ ]:
import os
os.chdir("/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_661")
adata.write("adata.h5ad")

In [ ]:
#run on tscc
#module load bedtools2
#cd /tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_662/outs
#/tscc/nfs/home/abhowell/create_cisTarget_databases/create_fasta_with_padded_bg_from_bed.sh /tscc/nfs/home/abhowell/cistarget_genomes/hg38.fa /tscc/nfs/home/abhowell/cistarget_genomes/hg38.chrom.sizes /tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_662/outs/consensus_peak_calling/consensus_regions.bed MM_662.with_1kb_bg_padding.fa 1000 yes

In [ ]:
#ls aertslab_motif_colleciton/v10nr_clust_public/singletons > motifs.txt
#mv motifs.txt cistarget_genomes/
#conda activate scenicplus
#conda install -c conda-forge python-flatbuffers
#have to modify line 301 to cluster_buster_path
#/tscc/nfs/home/abhowell/create_cisTarget_databases/create_cistarget_motif_databases.py -f /tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_662/outs/MM_662.with_1kb_bg_padding.fa -M /tscc/nfs/home/abhowell/aertslab_motif_colleciton/v10nr_clust_public/singletons -m /tscc/nfs/home/abhowell/cistarget_genomes/motifs.txt -o /tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_662/outs --bgpadding 1000 -t 20

In [ ]:
#import os
#os.chdir("/tscc/projects/ps-gaultonlab/projects/npod1/cellranger_output/Multiome/MM_664/outs/")

In [ ]:
#import mudata
#scplus_mdata = mudata.read("scplusmdata.h5mu")

In [ ]:
#scplus_mdata.uns["direct_e_regulon_metadata"]

In [ ]:
#scplus_mdata.uns["extended_e_regulon_metadata"]


In [ ]:
#import scanpy as sc
#import anndata
#eRegulon_gene_AUC = anndata.concat(
#    [scplus_mdata["direct_gene_based_AUC"], scplus_mdata["extended_gene_based_AUC"]],
#    axis = 1,
#)

#eRegulon_gene_AUC.obs = scplus_mdata.obs.loc[eRegulon_gene_AUC.obs_names]
#sc.pp.neighbors(eRegulon_gene_AUC, use_rep = "X")
#sc.tl.umap(eRegulon_gene_AUC)
#sc.pl.umap(eRegulon_gene_AUC, color = "scRNA_counts:celltype_assignment2")

In [ ]:
#from scenicplus.plotting.dotplot import heatmap_dotplot
#heatmap_dotplot(
#    scplus_mudata = scplus_mdata,
#    color_modality = "direct_gene_based_AUC",
#    size_modality = "direct_region_based_AUC",
#    group_variable = "scRNA_counts:celltype_assignment2",
#    eRegulon_metadata_key = "direct_e_regulon_metadata",
#    color_feature_key = "Gene_signature_name",
#    size_feature_key = "Region_signature_name",
#    feature_name_key = "eRegulon_name",
#    sort_data_by = "direct_gene_based_AUC",
#    orientation = "horizontal",
#    figsize = (16, 5)
#)